In [2]:
import numpy as np
import pandas as pd

In [90]:
data['title'].value_counts()[:10]

title
Terminator 3: Rise of the Machines    324
The Million Dollar Hotel              311
Solaris                               305
The 39 Steps                          291
Monsoon Wedding                       274
Once Were Warriors                    244
Three Colors: Red                     228
Men in Black II                       224
The Passion of Joan of Arc            218
Silent Hill                           215
Name: count, dtype: int64

In [17]:
data = pd.read_csv('final.csv')

In [18]:
matrix = data.pivot(index='userId', columns='movieId', values='rating').fillna(0)

In [19]:
matrix.head()

movieId,2,3,5,6,11,12,13,14,15,16,...,134158,134368,134569,134881,135536,140174,142507,148652,158238,160718
userId,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [20]:
from sklearn.metrics.pairwise import cosine_similarity

In [22]:
similarity_matrix = cosine_similarity(data.pivot(index='userId', columns='movieId', values='rating').fillna(0))

In [23]:
similarity_matrix

array([[1.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 1.        , 0.17289957, ..., 0.04433851, 0.17307257,
        0.10907781],
       [0.        , 0.17289957, 1.        , ..., 0.10607371, 0.22474113,
        0.20630672],
       ...,
       [0.        , 0.04433851, 0.10607371, ..., 1.        , 0.        ,
        0.12258009],
       [0.        , 0.17307257, 0.22474113, ..., 0.        , 1.        ,
        0.20027667],
       [0.        , 0.10907781, 0.20630672, ..., 0.12258009, 0.20027667,
        1.        ]])

In [30]:
def find_similar_users(user_id, top_n=5):
    # Calculating the similarity scores for all users compared to the target user
    similarity_scores = similarity_matrix[user_id]

    # similarity scores in descending order and get the indices
    most_similar_indices = np.argsort(similarity_scores)[::-1]

    # top N most similar users (excluding the target user itself)
    most_similar_users = most_similar_indices[1:top_n + 1]

    return most_similar_users


def recommend_movies(user_id, top_n=5):
    similar_users = find_similar_users(user_id, top_n)
    target_user_ratings = matrix.loc[user_id]

    # Extracting the movies that the target user has not rated
    unrated_movies = target_user_ratings[target_user_ratings == 0].index

    # Creatinng a table for similar users and their average ratings
    similar_users_ratings = matrix.loc[similar_users, unrated_movies]
    avg_ratings = similar_users_ratings.mean(axis=0)

    # Sorting the movies based on the average user ratings
    sorted_movies = avg_ratings.sort_values(ascending=False)

    # top N recommended movies and their average ratings
    top_movies = sorted_movies.index[:top_n]
    top_ratings = sorted_movies.values[:top_n]

    return top_movies, top_ratings

In [38]:
user_id = 43
top_n = 10
recommended_movies, ratings = recommend_movies(user_id, top_n)
print("Top recommended movies for user", user_id)
for movie_id, rating in zip(recommended_movies, ratings):
    movie_title = data.loc[data['movieId'] == movie_id, 'title'].values[0]
    print("Movie:", movie_title, "Average Rating:","{:.2f}".format(rating))

Top recommended movies for user 43
Movie: The Million Dollar Hotel Average Rating: 3.10
Movie: 5 Card Stud Average Rating: 2.70
Movie: Solaris Average Rating: 2.50
Movie: Say Anything... Average Rating: 2.30
Movie: The Tunnel Average Rating: 2.30
Movie: The Prisoner of Zenda Average Rating: 2.15
Movie: All the Way Boys Average Rating: 2.00
Movie: Terminator 3: Rise of the Machines Average Rating: 2.00
Movie: License to Wed Average Rating: 1.90
Movie: Batman Returns Average Rating: 1.85
